# BabyAGI Implementation with LLFn

<img src="https://user-images.githubusercontent.com/21254008/235015461-543a897f-70cc-4b63-941a-2ae3c9172b11.png" width="600"/>

In this example, we're going to rebuild [**BabyAGI**](https://github.com/yoheinakajima/babyagi#how-to-use) (originally created by
Yohei Nakajima) using [**LLFn**](https://github.com/orgexyz/LLFn).

### Modifications vs Original Implementation

- We will use almost exactly the same prompts as the original implementation (with minor modification for readability).
- We will NOT be using vectorstore to enrich the data, but instead including the past 5 tasks executed and their results for simplicity.

## **Before we start**: let's setup objective and keys for our BabyAGI

In [43]:
# Set objective here
OBJECTIVE = "Solve world hunger"

# Set API key here
OPENAI_MODEL = "gpt-3.5-turbo"
OPENAI_API_KEY = "<OPENAI_API_KEY>"

## **Step 1**: Installing dependencies and setup `function_prompt`

In [44]:
!pip install langchain llfn

In [45]:
import os
from llfn import LLFn
from pydantic import BaseModel, Field
from typing import List
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(
    temperature=0.7,
    model=OPENAI_MODEL,
    openai_api_key=OPENAI_API_KEY,
)

function_prompt = LLFn()
function_prompt.bind(llm)

## **Step 2**: Defining `task_creation_agent`

In [46]:
class TaskCreationOutput(BaseModel):
    new_tasks: List[str] = Field(..., description="a list of new tasks (may be empty)")
    reasoning: str = Field(..., description="why you think that way")

@function_prompt
def task_creation_agent(
    objective: str,
    last_task: str,
    last_task_result: str,
    task_list: List[str]
) -> TaskCreationOutput:
    return f"""
You are to use the result from an execution agent to create new tasks with the following objective: {objective}.
The last completed task has the result:
{last_task_result}
This result was based on this task description: {last_task}.

These are {len(task_list)} incomplete tasks: {', '.join(task_list)}

Based on the result, return a list of tasks to be completed in order to meet the objective.
These new tasks must not overlap with incomplete tasks.
"""

## **Step 3**: Defining `task_prioritization_agent`

In [47]:
class TaskPrioritizationOutput(BaseModel):
    tasks: List[str] = Field(..., description="a list of new tasks (may be empty)")
    reasoning: str = Field(..., description="why you prioritize tasks that way")

@function_prompt
def task_prioritization_agent(
    objective: str,
    task_list: List[str]
) -> TaskPrioritizationOutput:
    task_bullets = "\n".join([f"  - {task}" for task in task_list])
    return f"""
You are tasked with prioritizing the following tasks:
{task_bullets}

Consider the ultimate objective of your team: {objective}.
"""

## **Step 4**: Defining `task_execution_agent`

In [48]:
class TaskExecutionOutput(BaseModel):
    task: str = Field(..., description="task name")
    result: str = Field(..., description="result and effect of your execution and why")

@function_prompt
def task_execution_agent(
    objective: str,
    task: str,
    completed_tasks: List[TaskExecutionOutput]
) -> TaskExecutionOutput:
    completed_task_bullets = "\n".join([f"  - {execution.task}: {execution.result}" for execution in completed_tasks])
    return  f"""
Perform one task based on the following objective: {objective}.

Take into account these previously completed tasks:
{completed_task_bullets}

Your task: {task}
Response:
"""

## **Step 5**: Wire up everything and let BabyAGI run wild!

In [49]:
tasks = ["Develop a task list"]
completed_tasks = []

max_rounds = 5

for r in range(1, max_rounds+1):
    print(f"\n=====ROUND#{r}=====")

    # Print the task list
    print("\n*****TASK LIST*****")
    for t in tasks:
        print(f"- {t}")

    # Step 1: Pull the first incomplete task
    task = tasks.pop(0)
    print("\n*****NEXT TASK*****")
    print(task)

    # Send to execution function to complete the task based on the context
    execution = task_execution_agent(OBJECTIVE, task, completed_tasks[-5:])
    print("\n*****TASK RESULT*****")
    print(execution.result)

    # Step 2: Store the result in completed task
    completed_tasks.append(execution)

    # Step 3: Create new tasks re-prioritize task list
    new_tasks = task_creation_agent(OBJECTIVE, execution.task, execution.result, tasks).new_tasks

    print('\nAdding new tasks to task_storage')
    for t in new_tasks:
        print(f"- {t}")

    # Re-prioritize tasks
    tasks = task_prioritization_agent(OBJECTIVE, tasks + new_tasks).tasks

    # If no more tasks, terminate
    if not len(tasks):
        print("\nDone.")
        break



=====ROUND#1=====

*****TASK LIST*****
- Develop a task list

*****NEXT TASK*****
Develop a task list

*****TASK RESULT*****
To solve world hunger, we need to address various factors such as poverty, access to resources, agricultural practices, and distribution systems. Some possible tasks to work towards this objective include: 

1. Promote sustainable agriculture practices that increase food production and reduce environmental impact.
2. Implement social safety nets and poverty alleviation programs to ensure access to food for vulnerable populations.
3. Improve infrastructure and transportation systems to facilitate the distribution of food to remote areas.
4. Increase investment in research and development of innovative solutions for food production and preservation.
5. Collaborate with international organizations and governments to develop and implement comprehensive strategies to address world hunger.

These are just a few examples, and the specific tasks needed to solve world hu